In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [ ]:
import transformers
print(transformers.__version__)

4.15.0


### Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Reusing dataset wmt16 (/home/gabriel/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f)


  0%|          | 0/3 [00:00<?, ?it/s]

The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [ ]:
raw_datasets["train"][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'Also this week: the IMF approves a disbursement to Albania and Bulgaria sees a slowdown in industrial growth.', 'ro': 'Tot în această săptămână: FMI aprobă eliberarea unei tranşe pentru Albania, iar Bulgaria cunoaşte o încetinire a creşterii industriale.'}"
1,"{'en': '""The government has delivered on the promise.""', 'ro': '""Guvernul şi-a respectat promisiunea"".'}"
2,"{'en': 'I trust that it will not stop with today's debate and that we and the Commission shall together begin a process to implement the projects which you mentioned.', 'ro': 'Sper că acest lucru nu se va opri odată cu dezbaterea de azi şi că noi, împreună cu Comisia, vom începe un proces de desfăşurare a proiectelor pe care le-aţi menţionat.'}"
3,"{'en': 'Mr Bielan has just referred to the Services Directive, which can only function if professional qualifications are recognised by both sides.', 'ro': 'Dl Bielan tocmai a menţionat Directiva privind serviciile, care poate funcţiona doar în cazul în care calificările profesionale sunt recunoscute de ambele părţi.'}"
4,"{'en': 'I voted in favour of the report because I believe that strengthening the SOLVIT network can really help improve the legal framework of the internal market, which we are trying so hard to build.', 'ro': 'Am votat în favoarea raportului deoarece consider că o consolidare a reţelei SOLVIT poate ajuta la îmbunătăţirea cadrului juridic al pieţei interne, pe care încercăm din greu s-o construim.'}"


# Preprocessing the data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ro")

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[393, 4462, 14, 1137, 53, 216, 28636, 0], [24385, 14, 28636, 14, 4646, 4622, 53, 216, 28636, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[42140, 494, 1750, 53, 8, 59, 903, 3543, 9, 15202, 0], [36199, 6612, 9, 15202, 122, 568, 35788, 21549, 53, 8, 59, 903, 3543, 9, 15202, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/gabriel/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f/cache-77335284a4f61db6.arrow
Loading cached processed dataset at /home/gabriel/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f/cache-8f77f4c07614f0c6.arrow
Loading cached processed dataset at /home/gabriel/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/af3c5d746b307726d0de73ebe7f10545361b9cb6f75c83a1734c000e48b6264f/cache-8a23012c34f9112a.arrow


## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

checkpoint="opus-mt-en-it-finetuned-en-to-ro/checkpoint-762500"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)#Helsinki-NLP/opus-mt-en-it")

In [ ]:
batch_size = 16
model_name = "Helsinki-NLP/opus-mt-en-it".split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
#     output_dir='saved',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True    
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 610320
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 762900


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.943900,2.447408,8.453400,69.844400
2,0.870900,2.378713,8.690100,69.866400
3,0.824500,2.319946,8.862400,69.833900
4,0.782400,2.292705,9.007300,69.536300
5,0.802800,2.279715,9.156000,68.869400
6,0.750200,2.244400,9.177800,69.727900
7,0.739200,2.237357,9.211700,69.626800
8,0.709700,2.219377,9.174800,70.171600
9,0.688800,2.221941,9.165600,70.169100
10,0.672000,2.216024,9.301400,68.921500


Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-1000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-1000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-1000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-1000/special_tokens_m

Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-8000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-6500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-8500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-8500/special_to

Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-15000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-13500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-15500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-15500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-15500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-15500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-15500/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-14000] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-16000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-16000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkp

...

Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-739500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-739500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-739500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-739500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-739500/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-738000] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-740000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-740000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-740000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-740000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkp

tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-746500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-746500/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-745000] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-747000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-747000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-747000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-747000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-747000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-745500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-747500
Configuration saved in opus-mt-en-

Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-752000] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-754000
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-752500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-754500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-754500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/

Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761000/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761000/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-finetuned-en-to-ro/checkpoint-759500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-it-finetuned-en-to-ro/checkpoint-761500
Configuration saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761500/config.json
Model weights saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761500/tokenizer_config.json
Special tokens file saved in opus-mt-en-it-finetuned-en-to-ro/checkpoint-761500/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-it-f

TrainOutput(global_step=762900, training_loss=0.6852636821155296, metrics={'train_runtime': 108017.7938, 'train_samples_per_second': 113.004, 'train_steps_per_second': 7.063, 'total_flos': 2.017971782811648e+17, 'train_loss': 0.6852636821155296, 'epoch': 20.0})

Our fine tuned model already saved under *opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000*

Load the model and translate some text from english to romanian

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
src_text = [
    '''
    I met a traveller from an antique land,
    Who said—“Two vast and trunkless legs of stone
    Stand in the desert. . . . Near them, on the sand,
    Half sunk a shattered visage lies, whose frown,
    And wrinkled lip, and sneer of cold command,
    Tell that its sculptor well those passions read
    Which yet survive, stamped on these lifeless things,
    The hand that mocked them, and the heart that fed;
    And on the pedestal, these words appear:
    My name is Ozymandias, King of Kings;
    Look on my Works, ye Mighty, and despair!
    '''
]

model_name = 'weights/transformer_align'
# model_name = 'Helsinki-NLP/opus-mt-en-ro'

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True), 
                            num_beams=5)
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

Calculating BLEU score on EuroParl:

In [ ]:
import random
from tqdm import tqdm
import numpy as np 
from datasets import load_dataset, load_metric

with open('europarl-v7.ro-en.en') as f:
    en_lines = f.read().split("\n")[:-1]
with open('europarl-v7.ro-en.ro') as f:
    ro_lines = f.read().split("\n")[:-1]    
    
text_pairs = []
assert len(en_lines) == len(ro_lines)
for i in range(len(en_lines)):
    en = en_lines[i]
    ro = ro_lines[i]
    text_pairs.append((en, ro))

random.Random(4).shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
test_pairs = text_pairs[num_train_samples + num_val_samples :]

def batch(iterable, n=16):
    l = len(iterable)
    for ndx in tqdm(range(0, l // 3, n)):
        yield iterable[ndx:min(ndx + n, l)]

metric = load_metric("sacrebleu")

for pairs_batch in batch(test_pairs):
    eng_texts, ro_texts = zip(*pairs_batch)
    ro_texts = list(ro_texts)
    tk = tokenizer(eng_texts, return_tensors="pt", padding=True)['input_ids'].cuda()
    ro_preds_vec = model.generate(tk, num_beams=5) #.cuda()
    ro_preds = [tokenizer.decode(t, skip_special_tokens=True) for t in ro_preds_vec]
    for i in reversed(range(len(ro_texts))):
        if ro_texts[i] == None or ro_texts[i] == '':
            del ro_texts[i]
            del ro_preds[i]
    metric.add_batch(predictions=ro_preds, references=np.array(ro_texts).reshape(len(ro_texts), 1))
#     for i in range(16):
#         print(eng_texts[i])
#         print(ro_preds[i])
        
#     break
    
print(metric.compute())

Printing model schema:

In [ ]:
!pip install graphviz torchviz
from torchviz import make_dot

batch = tokenizer(src_text, return_tensors="pt", padding=True)['input_ids']
yhat = model(batch, decoder_input_ids=batch)['logits']
make_dot(yhat, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

Printing model parameter count:

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)